Start by importing the functions

In [1]:
from pathlib import Path

from xnemogcm import open_domain_cfg, open_nemo, open_namelist, open_nemo_and_domain_cfg
from xnemogcm import __version__ as xnemogcm_version

In [2]:
xnemogcm_version

'0.3.3'

# First open the domain_cfg and nemo files into 2 datasets
## domain

In [3]:
open_domain_cfg?

Signature: open_domain_cfg(datadir=None, files=None, add_coordinates=True)
Docstring:
Return a dataset containing all dataarrays of the domain_cfg*.nc / mesh_mask files.

For that, open and merge all the datasets.
The dataset is compatible with xgcm, the corresponding grid
can be create through: xgcm.Grid(domcfg)

Parameters
----------
datadir : string or pathlib.Path or None
    The directory containing the 'domain_cfg' or 'mesh_mask' files
files : list or iterator or None
    list of the file names that correspond to the domain_cfg and/or mesh_mask files,
    e.g. 'files=Path('path/to/data').glob('*my_domcfg*.nc') if your domain_cfg files are called
    'something_my_domcfg_00.nc' and 'something_my_domcfg_01.nc'
add_coordinates : bool
    Whether to add the 'glamt', 'gphit', etc as coordinates of the dataset

Returns
-------
domcfg : xarray.Dataset
    The domain configuration dataset, can be read by xgcm.
File:      ~/NEMO_UTILS/xnemogcm/xnemogcm/domcfg.py
Type:      function


You can provide the file names / folder using 3 similar methods:
1. Give the path to the files and xnemogcm opens the domain_cfg_out and mesh_mesk files
2. Give the path to the data folder + the name of the files
3. Give the name of the files that already contain the tree (e.g. ['/path/to/file1', '/path/to/file2']

These 3 methods are equivalent, however if your domain files don't have the strandard names you need to provide them by hand.

We use one of the test folder:

In [4]:
datadir = Path('../xnemogcm/test/data/open_and_merge/')

In [5]:
!ls ../xnemogcm/test/data/open_and_merge/

BASIN_grid_T.nc  BASIN_grid_V.nc  domain_cfg_out.nc
BASIN_grid_U.nc  BASIN_grid_W.nc


In [6]:
domcfg = open_domain_cfg(datadir=datadir)
# or
domcfg = open_domain_cfg(datadir=datadir, files=['domain_cfg_out.nc'])
# or
domcfg = open_domain_cfg(files=datadir.glob('*domain*.nc'))
domcfg

<xarray.Dataset>
Dimensions:       (x_c: 10, x_f: 10, y_c: 10, y_f: 10, z_c: 15, z_f: 15)
Coordinates: (12/16)
    glamt         (y_c, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    glamu         (y_c, x_f) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    glamv         (y_f, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    glamf         (y_f, x_f) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    gphit         (y_c, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    gphiu         (y_c, x_f) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    ...            ...
  * x_c           (x_c) int64 0 1 2 3 4 5 6 7 8 9
  * y_c           (y_c) int64 0 1 2 3 4 5 6 7 8 9
  * x_f           (x_f) float64 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5
  * y_f           (y_f) float64 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5
  * z_c           (z_c) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * z_f           (z_f) float64 -0.5 0.5 1.5 2.5 3.5 ... 9.5 10.5 11.5 12.5 13.5
Data variables: (12/35)
    nav_lon       (y_c, x_c) float32 dask.array<chunksize=(10, 10), meta=np.ndarray>
    nav_lat       (y_c, x_c) float32 dask.array<chunksize=(10, 10), meta=np.ndarray>
    jpiglo        int32 ...
    jpjglo        int32 ...
    jpkglo        int32 ...
    jperio        int32 ...
    ...            ...
    top_level     (y_c, x_c) int32 dask.array<chunksize=(10, 10), meta=np.ndarray>
    bottom_level  (y_c, x_c) int32 dask.array<chunksize=(10, 10), meta=np.ndarray>
    stiffness     (y_c, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    ht_0          (y_c, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    hu_0          (y_c, x_f) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    hv_0          (y_f, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
Attributes:
    DOMAIN_dimensions_ids:   [1 2]
    DOMAIN_size_global:      [10 10]
    DOMAIN_halo_size_start:  [0 0]
    DOMAIN_halo_size_end:    [0 0]
    DOMAIN_type:             BOX
    nn_cfg:                  4
    cn_cfg:                  BASIN

## Nemo

In [7]:
open_nemo?

Signature: open_nemo(domcfg, datadir=None, files=None, chunks=None, **kwargs_open)
Docstring:
Open nemo dataset, and rename the coordinates to be conform to xgcm.Grid

The filenames must finish with 'grid_X.nc', with X in
['T', 'U', 'V', 'W', 'UW', etc]

Parameters
----------
datadir : string or pathlib.Path
    The directory containing the nemo files
domcfg : xarray.Dataset
    the domcfg dataset, e.g. opened with xnemogcm.open_domain_cfg
files : list, optional
    List of the files to open
chunks : dict
    The chunks to use when opening the files,
    e.g. chunks={'time_counter':10}
    /! chunks need to be provided with the old names of dimensions
    i.e. 'time_counter', 'x', etc
    For more complex chunking, you may want to open without any chunks and set them up afterward.
kwargs_open : any other argument given to the xarray.open_mfdataset function
    e.g. parallel=True to use dask.delayed

Returns
-------
nemo_ds : xarray.Dataset
    Dataset containing all outputed variables,

We can provide the files folder / name following the same convention as for the `open_domain_cfg` function. We also need to provide the `domcfg` dataset so xnemogcm knows how to set the variables on the proper grid position. We can also provide extra kwargs to the underlying call to `xarray.open_mfdataset` function.

In [8]:
nemo = open_nemo(domcfg=domcfg, datadir=datadir)
# or
nemo = open_nemo(domcfg=domcfg, files=datadir.glob('*grid*.nc'))
nemo

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, t: 3, x_c: 10, x_f: 10, y_c: 10, y_f: 10, z_c: 15, z_f: 15)
Coordinates:
  * z_c                   (z_c) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
    time_centered         (t) object dask.array<chunksize=(3,), meta=np.ndarray>
  * t                     (t) object 0001-01-01 12:00:00 ... 0001-01-03 12:00:00
    time_instant          (t) object dask.array<chunksize=(3,), meta=np.ndarray>
  * x_c                   (x_c) int64 0 1 2 3 4 5 6 7 8 9
  * y_c                   (y_c) int64 0 1 2 3 4 5 6 7 8 9
  * x_f                   (x_f) float64 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5
  * y_f                   (y_f) float64 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.5 8.5 9.5
  * z_f                   (z_f) float64 -0.5 0.5 1.5 2.5 ... 10.5 11.5 12.5 13.5
Dimensions without coordinates: axis_nbounds
Data variables: (12/17)
    deptht_bounds         (z_c, axis_nbounds) float32 dask.array<chunksize=(15, 2), meta=np.ndarray>
    time_centered_bounds  (t, axis_nbounds) object dask.array<chunksize=(3, 2), meta=np.ndarray>
    t_bounds              (t, axis_nbounds) object dask.array<chunksize=(3, 2), meta=np.ndarray>
    time_instant_bounds   (t, axis_nbounds) object dask.array<chunksize=(3, 2), meta=np.ndarray>
    e3t                   (t, z_c, y_c, x_c) float32 dask.array<chunksize=(3, 15, 10, 10), meta=np.ndarray>
    thetao                (t, z_c, y_c, x_c) float32 dask.array<chunksize=(3, 15, 10, 10), meta=np.ndarray>
    ...                    ...
    depthv_bounds         (z_c, axis_nbounds) float32 dask.array<chunksize=(15, 2), meta=np.ndarray>
    e3v                   (t, z_c, y_f, x_c) float32 dask.array<chunksize=(3, 15, 10, 10), meta=np.ndarray>
    vo                    (t, z_c, y_f, x_c) float32 dask.array<chunksize=(3, 15, 10, 10), meta=np.ndarray>
    depthw_bounds         (z_f, axis_nbounds) float32 dask.array<chunksize=(15, 2), meta=np.ndarray>
    e3w                   (t, z_f, y_c, x_c) float32 dask.array<chunksize=(3, 15, 10, 10), meta=np.ndarray>
    woce                  (t, z_f, y_c, x_c) float32 dask.array<chunksize=(3, 15, 10, 10), meta=np.ndarray>
Attributes:
    name:         NEMO dataset
    description:  Ocean grid variables, set on the proper positions
    title:        Ocean grid variables
    Conventions:  CF-1.6
    timeStamp:    2021-Mar-12 17:10:24 GMT
    uuid:         8290615f-bfc5-46b0-8a9f-91007a66082f

# Open both at once

It is possible to open the domain and nemo output at once in one unique dataset. What happens is that 2 datasets are created and then merged. Thus all option possible for the `open_nemo` and `open_domain_cfg` functions are still possible.

In [9]:
open_nemo_and_domain_cfg?

Signature:
open_nemo_and_domain_cfg(
    nemo_files=None,
    domcfg_files=None,
    nemo_kwargs=None,
    domcfg_kwargs=None,
    linear_free_surface=False,
)
Docstring:
Open nemo_ds and domcfg with open_nemo and open_domain_cfg and merge them with _merge_nemo_and_domain_cfg.

See the respective functions docstrings for more details.

2 methods are available for nemo files and domain_cfg/mesh_mask files: 1) provide a list of the files
you want to open, 2) provide the path of the directories containing the files and xnemogcm will try
to open as much files as it can.

Arguments
---------
nemo_files : Optional, list / generator or string / Path
    1) list / generator containing the nemo output files, or
    2) string / Path of the directory containing the nemo output files.
       Will open all files containing "grid_X" in their name, "X" being "T", "U", "V", "W", "F", etc
domcfg_files : Optional, list / generator or string / Path
    1) list / generator containing the domain_cfg / mesh

Again, mutiple equivalent arguments are possible to open the data

In [10]:
# the simplest for simple cases, provide the path
ds = open_nemo_and_domain_cfg(nemo_files=datadir, domcfg_files=datadir)
# or provide the files
ds = open_nemo_and_domain_cfg(nemo_files=datadir.glob('*grid*.nc'), domcfg_files=datadir.glob('*domain*.nc'))
# or use the nemo_kwargs and domcfg_kwargs dictionnaries
ds = open_nemo_and_domain_cfg(nemo_kwargs=dict(datadir=datadir), domcfg_kwargs={'files':datadir.glob('*domain*.nc')})
ds

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, t: 3, x_c: 10, x_f: 10, y_c: 10, y_f: 10, z_c: 15, z_f: 15)
Coordinates: (12/19)
  * z_c                   (z_c) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
    time_centered         (t) object dask.array<chunksize=(3,), meta=np.ndarray>
  * t                     (t) object 0001-01-01 12:00:00 ... 0001-01-03 12:00:00
    time_instant          (t) object dask.array<chunksize=(3,), meta=np.ndarray>
  * x_c                   (x_c) int64 0 1 2 3 4 5 6 7 8 9
  * y_c                   (y_c) int64 0 1 2 3 4 5 6 7 8 9
    ...                    ...
    gphit                 (y_c, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    gphiu                 (y_c, x_f) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    gphiv                 (y_f, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    gphif                 (y_f, x_f) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    gdept_0               (z_c, y_c, x_c) float64 dask.array<chunksize=(15, 10, 10), meta=np.ndarray>
    gdepw_0               (z_f, y_c, x_c) float64 dask.array<chunksize=(15, 10, 10), meta=np.ndarray>
Dimensions without coordinates: axis_nbounds
Data variables: (12/52)
    deptht_bounds         (z_c, axis_nbounds) float32 dask.array<chunksize=(15, 2), meta=np.ndarray>
    time_centered_bounds  (t, axis_nbounds) object dask.array<chunksize=(3, 2), meta=np.ndarray>
    t_bounds              (t, axis_nbounds) object dask.array<chunksize=(3, 2), meta=np.ndarray>
    time_instant_bounds   (t, axis_nbounds) object dask.array<chunksize=(3, 2), meta=np.ndarray>
    e3t                   (t, z_c, y_c, x_c) float32 dask.array<chunksize=(3, 15, 10, 10), meta=np.ndarray>
    thetao                (t, z_c, y_c, x_c) float32 dask.array<chunksize=(3, 15, 10, 10), meta=np.ndarray>
    ...                    ...
    top_level             (y_c, x_c) int32 dask.array<chunksize=(10, 10), meta=np.ndarray>
    bottom_level          (y_c, x_c) int32 dask.array<chunksize=(10, 10), meta=np.ndarray>
    stiffness             (y_c, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    ht_0                  (y_c, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    hu_0                  (y_c, x_f) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    hv_0                  (y_f, x_c) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
Attributes: (12/13)
    name:                    NEMO dataset
    description:             Ocean grid variables, set on the proper positions
    title:                   Ocean grid variables
    Conventions:             CF-1.6
    timeStamp:               2021-Mar-12 17:10:24 GMT
    uuid:                    8290615f-bfc5-46b0-8a9f-91007a66082f
    ...                      ...
    DOMAIN_size_global:      [10 10]
    DOMAIN_halo_size_start:  [0 0]
    DOMAIN_halo_size_end:    [0 0]
    DOMAIN_type:             BOX
    nn_cfg:                  4
    cn_cfg:                  BASIN

## Remark

All opening are lazy using dask, which makes files quick to open, until you actually load the data you need

# Namelist

It can be convenient to open the namelist used for the run (e.g. to compare different runs with different parameters). This is possible using the `f90nml` package (it needs to be installed, this is an optional dependency).

In [11]:
open_namelist?

Signature: open_namelist(datadir=None, files=None, ref=True, cfg=True)
Docstring: Open the namelist and store it into a xarray.Dataset
File:      ~/NEMO_UTILS/xnemogcm/xnemogcm/namelist.py
Type:      function


Here you provide the folder path containing the reference and configuration namelists, or the filenames (as for nemo and domcfg). You can choose to load both, or only one of them. The configration namelist will overwrite the default one.

For this we need to use another folder of the test data (with simplified namelists for the exemple):

In [12]:
datadir = Path('../xnemogcm/test/data/namelist/')

In [13]:
!ls ../xnemogcm/test/data/namelist/

namelist_cfg  namelist_ref


In [14]:
nam = open_namelist(datadir)
nam

<xarray.Dataset>
Dimensions:    ()
Data variables:
    nn_no      int64 0
    cn_exp     <U5 'BASIN'
    nn_it000   int64 1
    nn_itend   int64 5840
    rn_e1_deg  float64 1.0

In [15]:
nam.nn_it000

<xarray.DataArray 'nn_it000' ()>
array(1)
Attributes:
    namelist:  namrun